In [1]:
import sys
print("Python version:", sys.version)

import pandas as pd
import sympy
import openai

print("pandas version:", pd.__version__)
print("sympy version:", sympy.__version__)
print("openai version:", openai.__version__)

Python version: 3.11.13 (main, Jun  5 2025, 08:21:08) [Clang 14.0.6 ]
pandas version: 2.2.3
sympy version: 1.14.0
openai version: 1.109.1


In [2]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.responses.create(
  model="gpt-3.5-turbo",
  input="write a haiku about ai",
  store=True,
)

print(response.output_text);

Artificial minds,
Computing with lightning speed,
AI's future bright.


In [3]:
problem_text1 = """
Solve this Math Magazine problem:

You are given a vertical line that starts at the origin and extends upwards infinitely.
You are given a point (p, q)
You are given constance \alpha \in [0, 1] and G \in \mathbb{R}

You want to draw a curve connecting (p, q) to some point on the vertical line. You want to optimize the following:
- the wiring cost is the length of the curve from (p, q) to the vertical line
- the conduction delay is the length of the curve from (p, q) to the vertical line + the distance fron the connection point on the vertial line to the origin
- You want to minimize \alpha * wiring + (1 - \alpha) * delay

You are subject to the constraint that your curve must follow a parabola whose quadratic coefficient is G
"""

In [6]:
try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # start with GPT-3.5 for testing
        messages=[{"role": "user", "content": problem_text1}],
        temperature=0,           # deterministic output
        max_tokens=500           # adjust for expected solution length
    )
    model_output = response.choices[0].message.content
except Exception as e:
    model_output = f"Error: {e}"

In [7]:
model_output

"To solve this problem, we can first parameterize the curve using the equation of a parabola:\n\ny = ax^2 + bx + c\n\nSince the curve must pass through the point (p, q), we have the following equations:\n\nq = ap^2 + bp + c\n0 = a(0)^2 + b(0) + c\n\nFrom the second equation, we get c = 0. Substituting this into the first equation, we get:\n\nq = ap^2 + bp\n\nNow, we need to find the point on the vertical line that minimizes the cost function. Let's denote this point as (0, y). The distance from (p, q) to (0, y) is given by:\n\ndistance = sqrt((p-0)^2 + (q-y)^2)\n         = sqrt(p^2 + (q-y)^2)\n\nThe distance from (0, y) to the origin is simply y. Therefore, the conduction delay is:\n\ndelay = sqrt(p^2 + (q-y)^2) + y\n\nThe wiring cost is just the distance from (p, q) to (0, y), which is:\n\nwiring = sqrt(p^2 + (q-y)^2)\n\nNow, we can write the cost function as:\n\ncost = alpha * wiring + (1-alpha) * delay\n     = alpha * sqrt(p^2 + (q-y)^2) + (1-alpha) * (sqrt(p^2 + (q-y)^2) + y)\n\nTo

In [9]:
df = pd.DataFrame([{
    "prompt_text": problem_text1,
    "model_output": model_output,
    "score": 0,   # Fill manually: "Correct", "Partial", "Incorrect"
    "notes": "describes approach but does not solve the problem"    # Optional: reasoning errors, hallucinations
}])

output_file = "gpt_math_eval.csv"
df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

# ----------------------------
# 6. Display table
# ----------------------------
df

Results saved to gpt_math_eval_one_prompt.csv


,prompt_text,model_output,score,notes
0,\nSolve this Math Magazine problem:\n\nYou are...,"To solve this problem, we can first parameteri...",0,describes approach but does not solve the problem


In [20]:
TEMPERATURE = 0.1
MAX_TOKENS = 1000

In [21]:
def evaluate_prompt(prompt_text, temperature=TEMPERATURE, max_tokens=MAX_TOKENS):
    """
    Runs a prompt through GPT, prints output, asks for manual score & feedback (0/0.5/1),
    and appends a row to a CSV file.
    """
    # Run GPT
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt_text}],
            temperature=0,
            max_tokens=500
        )
        model_output = response.choices[0].message.content
    except Exception as e:
        model_output = f"Error: {e}"

    # Print the output
    print("\n--- GPT Output ---")
    print(model_output)
    print("------------------\n")

    # Ask for score
    valid_scores = ["0", "0.5", "1"]
    score = None
    while score not in valid_scores:
        score = input("Enter score (0 / 0.5 / 1): ").strip()
        if score not in valid_scores:
            print("Invalid score. Please enter 0, 0.5, or 1.")

    # Ask for feedback
    feedback = input("Enter feedback / notes: ").strip()

    # Append to CSV
    df_new = pd.DataFrame([{
        "prompt_text": prompt_text,
        "model_output": model_output,
        "score": float(score),
        "feedback": feedback
    }])
    
    output_file = "gpt_math_eval.csv"
    df_new.to_csv(output_file, mode='a', header=False, index=False)
    print(f"Entry saved to {output_file}\n")

In [14]:
problem_text2 = """
Solve this Math Magazine problem:

You are given a vertical line that starts at the origin and extends upwards infinitely.
You are given a point (p, q)
You are given constance \alpha \in [0, 1] and G \in \mathbb{R}
0
You want to draw a curve connecting (p, q) to some point on the vertical line. You want to optimize the following:
- the wiring cost is the length of the curve from (p, q) to the vertical line
- the conduction delay is the length of the curve from (p, q) to the vertical line + the distance fron the connection point on the vertial line to the origin
- You want to minimize \alpha * wiring + (1 - \alpha) * delay

You are subject to the constraint that your curve must follow a parabola whose quadratic coefficient is G

Output a formula for the optimal point (0, t) in terms of p, q, G, \alpha
"""

In [15]:
evaluate_prompt(problem_text2)


--- GPT Output ---
To solve this problem, we need to find the equation of the parabola that satisfies the given constraints. 

Let the equation of the parabola be y = ax^2 + bx + c, where a = G, b = 0 (since the parabola is symmetric about the y-axis), and c is the y-intercept. 

Since the parabola passes through the point (p, q), we have q = ap^2 + c. 

Also, the derivative of the parabola at point (p, q) should be equal to the slope of the line connecting (p, q) to the vertical line. 

The slope of the line connecting (p, q) to the vertical line is given by (q - t)/(p - 0) = (q - t)/p. 

The derivative of the parabola at point (p, q) is given by 2ap. 

Setting these two equal, we get 2ap = (q - t)/p. 

Solving for t, we get t = q - 2ap^2. 

Therefore, the optimal point (0, t) in terms of p, q, G, and lpha is (0, q - 2a*p^2), where a = G.
------------------



Enter score (0 / 0.5 / 1):  0
Enter feedback / notes:  Wrongly assumes b = 0 instead of solving for b. appears to assume a line? Alpha is not part of the solution


Entry saved to gpt_math_eval_one_prompt.csv



In [18]:
problem_text3 = """
Solve this Math Magazine problem:

You are given a vertical line that starts at the origin and extends upwards infinitely.
You are given a point (p, q)
You are given constance \alpha \in [0, 1] and G \in \mathbb{R}
0
You want to draw a curve connecting (p, q) to some point on the vertical line. You want to optimize the following:
- the wiring cost is the length of the curve from (p, q) to the vertical line
- the conduction delay is the length of the curve from (p, q) to the vertical line + the distance fron the connection point on the vertial line to the origin
- You want to minimize \alpha * wiring + (1 - \alpha) * delay

You are subject to the following constraints: 
- that your curve must follow a parabola of the form y(x) = ax^2 + bx + c
- a = G
- y(0) = t
- y(p) = q

Output a formula for the optimal point (0, t) in terms of p, q, G, \alpha

Here are some hints:
- You will need to use the well-known formula for the length of a curve
- you may want to define an intermediate function b(t), as the choice of t constrains the linear coefficient in the quadratic curve
- An analytic solution requires using leibniz's rule as well as the fundamental theorem of calculus
"""

In [19]:
evaluate_prompt(problem_text3)


--- GPT Output ---
To find the optimal point (0, t) in terms of p, q, G, and lpha, we need to minimize the function lpha * wiring + (1 - lpha) * delay.

Let's start by defining the parabolic curve y(x) = ax^2 + bx + c. Since a = G, the curve becomes y(x) = Gx^2 + bx + c.

Given that y(0) = t, we have c = t.

Given that y(p) = q, we have Gp^2 + bp + t = q.

Now, let's define the length of the curve from (p, q) to the vertical line as L = ∫[0, p] √(1 + (dy/dx)^2) dx.

The derivative of y(x) = Gx^2 + bx + t is dy/dx = 2Gx + b.

Therefore, the length L becomes L = ∫[0, p] √(1 + (2Gx + b)^2) dx.

Next, let's define the conduction delay D as the length of the curve from (p, q) to the vertical line plus the distance from the connection point on the vertical line to the origin. Therefore, D = L + √(p^2 + t^2).

Now, we want to minimize lpha * L + (1 - lpha) * D with respect to t.

To find the optimal t, we need to take the derivative of the above expression with respect to t, set it equa

Enter score (0 / 0.5 / 1):  0
Enter feedback / notes:  Does not attempt to solve it


Entry saved to gpt_math_eval.csv



In [22]:
problem_text4 = """
Solve this Math Magazine problem:

You are given a vertical line that starts at the origin and extends upwards infinitely.
You are given a point (p, q)
You are given constance \alpha \in [0, 1] and G \in \mathbb{R}
0
You want to draw a curve connecting (p, q) to some point on the vertical line. You want to optimize the following:
- the wiring cost is the length of the curve from (p, q) to the vertical line
- the conduction delay is the length of the curve from (p, q) to the vertical line + the distance fron the connection point on the vertial line to the origin
- You want to minimize \alpha * wiring + (1 - \alpha) * delay

You are subject to the following constraints: 
- that your curve must follow a parabola of the form y(x) = ax^2 + bx + c
- a = G
- y(0) = t
- y(p) = q

Output a formula for the optimal point (0, t) in terms of p, q, G, \alpha. Give me output that I can plug into sympy to check your work.

Here are some hints:
- You will need to use the well-known formula for the length of a curve
- you may want to define an intermediate function b(t), as the choice of t constrains the linear coefficient in the quadratic curve
- One analytic solution uses leibniz's rule as well as the fundamental theorem of calculus.
"""

In [23]:
evaluate_prompt(problem_text4)


--- GPT Output ---
To solve this problem, we need to find the optimal point (0, t) on the vertical line that minimizes the given cost function. Let's denote the curve as y(x) = ax^2 + bx + c.

Given constraints:
- a = G
- y(0) = t
- y(p) = q

We can substitute these constraints into the equation of the curve to get:
c = t
Gp^2 + bp + t = q

Now, we need to find the optimal value of t that minimizes the cost function. The cost function is given by:
Cost = lpha * wiring + (1 - lpha) * delay

The wiring cost is the length of the curve from (p, q) to the vertical line, which can be calculated using the formula for the length of a curve:
wiring = ∫[p, 0] √(1 + (dy/dx)^2) dx

The conduction delay is the length of the curve from (p, q) to the vertical line plus the distance from the connection point on the vertical line to the origin:
delay = ∫[p, 0] √(1 + (dy/dx)^2) dx + √(p^2 + t^2)

To find the optimal value of t, we need to minimize the cost function by taking the derivative of the cos

Enter score (0 / 0.5 / 1):  0
Enter feedback / notes:  still does not attempt to solve it on its own


Entry saved to gpt_math_eval.csv



In [24]:
problem_text5 = """
You are an expert in symbolic math and calculus. 
Solve the following Math Magazine problem fully and symbolically, and give output that can be copied directly into Python/Sympy. 
Do not provide prose explanation, only define the expressions.

Problem:

- You have a vertical line starting at the origin, and a point (p, q)
- A constant alpha in [0,1] and G in R
- You want to draw a curve connecting (p,q) to some point (0,t) on the vertical line
- The curve must follow y(x) = G*x^2 + b*x + t
- Constraints: y(p) = q
- Objective: minimize alpha*wiring + (1-alpha)*delay
  - wiring = length of the curve from (p,q) to (0,t)
  - delay = wiring + distance from connection to origin
- Use the Leibniz rule and fundamental theorem of calculus to differentiate the integral
- Define b(t) = (q - t - G*p**2)/p
- Solve d(Objective)/dt = 0 for t
- Output: a Python/Sympy expression for t in terms of p, q, G, alpha

Only output valid Python/Sympy code. Do not include explanations.
"""

In [25]:
evaluate_prompt(problem_text5)


--- GPT Output ---
from sympy import symbols, diff, solve

p, q, G, alpha, t = symbols('p q G alpha t')
b = (q - t - G*p**2)/p
wiring = 2*G*p**2 + p*t
delay = wiring + t
Objective = alpha*wiring + (1-alpha)*delay

dObjective_dt = diff(Objective, t)
t_solution = solve(dObjective_dt, t)[0]

t_solution
------------------



Enter score (0 / 0.5 / 1):  0
Enter feedback / notes:  doesn't give its solution


Entry saved to gpt_math_eval.csv

